In [1]:
MODEL = "llama2"

In [2]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings


model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17412\1266271018.py:5: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model=MODEL)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17412\1266271018.py:6: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=MODEL)


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, chunk_overlap=0, add_start_index=False
)


In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document

In [5]:
loader = TextLoader(r"C:\Users\DELL\application\techathon\chatBot_data\introduction-to-stock-markets\key-events-and-their-impact-on-markets.txt", encoding='utf-8')

In [6]:
pages = loader.load_and_split()

In [7]:
all_splits = text_splitter.split_documents(pages)

len(all_splits)

98

In [9]:
from langchain_community.vectorstores import DocArrayInMemorySearch
db = DocArrayInMemorySearch.from_documents(pages, embeddings)

In [11]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()


In [12]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [14]:
retriever = db.as_retriever()

In [15]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [16]:
def chatbot(question):
    # Retrieve context from the vector store
    context = db.similarity_search(question, k=1)[0].page_content

    # Format the prompt with context and question
    prompt_text = prompt.format(context=context, question=question)

    # Get the response from the model
    model_response = model.invoke(prompt_text)
    response = parser.parse(model_response)
    return response

In [17]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    response = chatbot(user_input)
    print(f"Bot: {response}")

You:  what is the impact of stock market?


Bot: The impact of a high stock market on inflation can be both positive and negative. On the one hand, a booming stock market can lead to increased investment and economic growth, which can help reduce inflation by increasing productivity and output. On the other hand, a stock market bubble can lead to an overvaluation of assets, which can eventually burst and lead to a decline in the value of money, potentially causing higher inflation.

In general, the impact of the stock market on inflation is complex and depends on various factors such as the state of the economy, monetary policy, and investor sentiment. However, research suggests that there is a positive relationship between stock market performance and inflation, meaning that when the stock market performs well, inflation tends to be lower, and vice versa.

For example, a study by Harvey et al. (2017) found that in developed economies, a strong stock market is associated with lower inflation. Another study by Bodenhorn and Hodge

You:  what is stock market?


Bot:  Based on the context provided, the answer to the question "What is a stock market?" can be inferred as follows:

A stock market is a platform where companies can issue and trade shares of their stocks. It provides a means for investors to buy and sell ownership stakes in these companies, thereby enabling them to raise capital and access liquidity. The stock market is an essential component of a country's financial system, as it allows businesses to access funding from a wide range of investors and provides investors with opportunities to grow their wealth through investments in equities.

In the context provided, the term "stock market" is not explicitly mentioned, but it can be inferred based on the discussion around inflation, interest rates, and industrial production. The passage explains how inflation can impact the purchasing power of money and how central banks like RBI work to reduce inflation to a manageable level. It also mentions the importance of industrial production 

You:  what is  Monetary Policy ?


Bot: Monetary policy refers to the actions taken by a central bank or other monetary authority to manage a country's money supply and interest rates to promote economic growth, stability, and low inflation. The goal of monetary policy is to ensure that the money supply and interest rates are at levels that support the economy's growth and stability, while keeping inflation under control.

Monetary policy tools include:

1. Interest Rates: The central bank can set interest rates to influence borrowing and spending habits of individuals and businesses. Lower interest rates can stimulate economic activity, while higher interest rates can slow it down.
2. Money Supply: The central bank can control the money supply by adjusting the amount of money it prints or removes from circulation. An increase in the money supply can boost economic growth, while a decrease can reduce inflation.
3. Forward Guidance: The central bank can communicate its future policy intentions to influence market expecta

You:  what is monetary policy in story telling way?


Bot: In the context of the given passage, "monetary policy" refers to the actions taken by a central bank to manage the money supply and interest rates in an economy. The central bank uses monetary policy tools such as interest rates and money creation to influence economic activity, stabilize prices, and achieve macroeconomic objectives.

In the storytelling way, monetary policy can be thought of as a superhero who fights inflation and maintains economic stability. Just like how a superhero uses their powers to protect the city from villains, the central bank uses monetary policy tools to protect the economy from inflation and other macroeconomic threats.

For example, when inflation is rising, the central bank can use interest rate hikes as its "superpower" to slow down the money supply growth and reduce inflationary pressures. Similarly, when the economy is facing a recession, the central bank can use monetary policy tools to inject liquidity into the system and stimulate economic g

You:  exit


In [18]:
story_telling =  """
Answer the question in story telling way based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

In [19]:
prompt2 = PromptTemplate.from_template(story_telling)

In [21]:
from operator import itemgetter

chain2 = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt2
    | model
    | parser
)

In [22]:
def Story_chatbot(question):
    # Retrieve context from the vector store
    context = db.similarity_search(question, k=1)[0].page_content

    # Format the prompt with context and question
    prompt_text = prompt2.format(context=context, question=question)

    # Get the response from the model
    model_response = model.invoke(prompt_text)
    response = parser.parse(model_response)
    return response

In [24]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    response = chatbot(user_input)
    print(f"Bot: {response}")

You:  exit
